In [43]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['fair-classification']


In [44]:
print(os.listdir('../input/fair-classification'))

['test_no_income.csv', 'test_sample0.csv', 'train.csv']


In [45]:
train_df = pd.read_csv('../input/fair-classification/train.csv')
test_df = pd.read_csv('../input/fair-classification/test_no_income.csv')

In [46]:
train_df.head()

,Age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital gain,capital loss,hours per week,native-country,income
0,44,Local-gov,100479,Assoc-acdm,12,Married-civ-spouse,Prof-specialty,Husband,0,0,0,0,48,United-States,0
1,57,Private,477867,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,0,0,0,0,40,United-States,0
2,21,Private,410439,Some-college,10,Never-married,Sales,Own-child,0,0,0,0,24,United-States,0
3,21,Private,195532,HS-grad,9,Never-married,Adm-clerical,Unmarried,0,1,0,0,40,United-States,0
4,57,Self-emp-inc,125000,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,0,0,0,0,35,United-States,1


In [47]:
test_df.head()

,Age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital gain,capital loss,hours per week,native-country
0,56,Local-gov,264436,Some-college,10,Divorced,Exec-managerial,Not-in-family,0,1,0,0,40,United-States
1,37,Private,184556,Some-college,10,Divorced,Tech-support,Unmarried,0,1,0,0,40,United-States
2,64,Local-gov,96076,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,0,0,0,0,40,United-States
3,36,Federal-gov,192150,HS-grad,9,Divorced,Adm-clerical,Unmarried,0,1,0,0,40,United-States
4,30,?,147215,Some-college,10,Never-married,?,Own-child,0,1,0,0,30,United-States


In [48]:
test_df['workclass'].unique()

array(['Local-gov', 'Private', 'Federal-gov', '?', 'Self-emp-not-inc',
       'Self-emp-inc', 'State-gov', 'Never-worked', 'Without-pay'],
      dtype=object)

In [49]:
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)

In [51]:
test_df.columns

Index(['Age', 'fnlwgt', 'education-num', 'race', 'gender', 'capital gain',
       'capital loss', 'hours per week', 'workclass_?',
       'workclass_Federal-gov', 'workclass_Local-gov',
       'workclass_Never-worked', 'workclass_Private', 'workclass_Self-emp-inc',
       'workclass_Self-emp-not-inc', 'workclass_State-gov',
       'workclass_Without-pay', 'education_10th', 'education_11th',
       'education_12th', 'education_1st-4th', 'education_5th-6th',
       'education_7th-8th', 'education_9th', 'education_Assoc-acdm',
       'education_Assoc-voc', 'education_Bachelors', 'education_Doctorate',
       'education_HS-grad', 'education_Masters', 'education_Preschool',
       'education_Prof-school', 'education_Some-college',
       'marital-status_Divorced', 'marital-status_Married-AF-spouse',
       'marital-status_Married-civ-spouse',
       'marital-status_Married-spouse-absent', 'marital-status_Never-married',
       'marital-status_Separated', 'marital-status_Widowed', 'occupatio

In [52]:
missing_cols = set(train_df.columns) - set(test_df.columns)

for c in missing_cols:
    if c != 'income':
        test_df[c] = 0
    
# Ensure Ordering matches
test_df = test_df[train_df.columns]

native-country_Hong
native-country_South
native-country_Vietnam
income
native-country_Holand-Netherlands


In [11]:
train_df.columns == test_df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [12]:
X_train, y_train = train_df.drop('income', axis=1), train_df['income']
X_test = test_df

In [13]:
from sklearn.ensemble import RandomForestClassifier

# Training
rf = RandomForestClassifier(n_estimators=1000, random_state=42, verbose=1)
rf.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  1.8min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=42, verbose=1, warm_start=False)

In [14]:
# Make Predictions
predictions = rf.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    5.3s finished


In [15]:
# Determine absolute error
errors = abs(predictions - y_test)

# Determine number of errors
num_correct = errors.value_counts()[0]
num_incorrect = errors.value_counts()[1]


# Determine accuracy
accuracy = (1 - (num_incorrect / num_correct)) * 100

print('Accuracy: ', round(accuracy, 2), '%.')

Accuracy:  73.55 %.


In [17]:
# Determine Bias
Th = 0.1
# DDP = np.abs()

In [34]:
A0_indices = X_test.loc[X_test['gender'] == 0].index.values
A1_indices = X_test.loc[X_test['gender'] == 1].index.values

total_num_A0 = len(A0_indices)
total_num_A1 = len(A1_indices)

N1A0 = y_test.iloc[A0_indices]
N1A0.unique()

array([0], dtype=int64)

In [29]:
# Evaluate

importances = list(rf.feature_importances_)
feature_importances = [(feature, round(importance, 5)) for feature, importance in zip(X_train.columns, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse=True)
list_of_importances = [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: fnlwgt               Importance: 0.17186
Variable: Age                  Importance: 0.15397
Variable: capital gain         Importance: 0.09766
Variable: hours per week       Importance: 0.08556
Variable: marital-status_Married-civ-spouse Importance: 0.06519
Variable: education-num        Importance: 0.06243
Variable: relationship_Husband Importance: 0.04555
Variable: capital loss         Importance: 0.03049
Variable: marital-status_Never-married Importance: 0.02375
Variable: occupation_Exec-managerial Importance: 0.01841
Variable: occupation_Prof-specialty Importance: 0.01528
Variable: education_Bachelors  Importance: 0.01239
Variable: gender               Importance: 0.01226
Variable: relationship_Not-in-family Importance: 0.01073
Variable: workclass_Private    Importance: 0.00969
Variable: relationship_Wife    Importance: 0.0095
Variable: relationship_Own-child Importance: 0.00855
Variable: workclass_Self-emp-not-inc Importance: 0.00827
Variable: education_Masters    Import

In [32]:
import datetime

file_name = '../output/test_sample_' + str(datetime.datetime.now()).replace(' ', '').replace(':','') + '.csv'
file_name

'../output/test_sample_2019-04-09224929.697337.csv'

In [39]:
submission = pd.Series(predictions)
submission

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      1
18      0
19      0
20      0
21      1
22      0
23      1
24      0
25      1
26      0
27      0
28      0
29      1
       ..
8967    1
8968    0
8969    0
8970    0
8971    1
8972    0
8973    0
8974    0
8975    0
8976    0
8977    0
8978    1
8979    1
8980    1
8981    0
8982    0
8983    0
8984    0
8985    1
8986    1
8987    0
8988    0
8989    1
8990    0
8991    1
8992    0
8993    0
8994    0
8995    0
8996    0
Length: 8997, dtype: int64

In [45]:
submission.to_csv(file_name, index_label='Id', header=['income'], index=True)